In [2]:
using GLM
using StatsBase
using DataFrames
using Dagitty
using Turing
using StatsPlots
using StatisticalRethinking
using Logging

default(labels=false)
Logging.disable_logging(Logging.Warn);

# 7.1 The problem with parameters

Code 7.1

In [3]:
sppnames = ["afarensis", "africanus", "habilis", "boisei", "rudolfensis", "ergaster", "sapiens"]
brainvolcc = [438, 452, 612, 521, 752, 871, 1350]
masskg = [37.0, 35.5, 34.5, 41.5, 55.5, 61.0, 53.5]
d = DataFrame(:species => sppnames, :brain => brainvolcc, :mass => masskg);

Code 7.2

In [12]:
d[!,:mass_std] = (d.mass .- mean(d.mass))./std(d.mass)
d[!,:brain_std] = d.brain ./ maximum(d.brain);

Code 7.3

In [14]:
@model function model_m7_1(mass_std, brain_std)
    a ~ Normal(0.5, 1)
    b ~ Normal(0, 10)
    μ = @. a + b*mass_std
    log_σ ~ Normal()
    brain_std ~ MvNormal(μ, exp(log_σ))
end

m7_1_ch = sample(model_m7_1(d.mass_std, d.brain_std), NUTS(), 1000)
m7_1 = DataFrame(m7_1_ch)
precis(m7_1)

┌───────┬────────────────────────────────────────────────────────┐
│ param │    mean     std     5.5%      50%   94.5%    histogram │
├───────┼────────────────────────────────────────────────────────┤
│     a │  0.5259  0.1093    0.344    0.524  0.6854  ▁▁▁▂▆█▄▁▁▁▁ │
│     b │  0.1652   0.121   0.0045   0.1653  0.3346     ▁▁▁▁█▅▁▁ │
│ log_σ │ -1.3829  0.3567  -1.8915  -1.4062  -0.791       ▁▆█▂▁▁ │
└───────┴────────────────────────────────────────────────────────┘


Code 7.4

In [17]:
X = hcat(ones(length(d.mass_std)), d.mass_std)
m = lm(X, d.brain_std)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
───────────────────────────────────────────────────────────────
       Coef.  Std. Error     t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────
x1  0.528677   0.0705692  7.49    0.0007   0.347273    0.710081
x2  0.167118   0.0762235  2.19    0.0798  -0.0288204   0.363057
───────────────────────────────────────────────────────────────
